In [45]:
import ROOT
import numpy as np
import os

In [46]:
for i in [15,18,21,24,27,30,35]:
    input_dir = f"/home/master/Desktop/443exps/x-ray/data/{i}kV-XRay.txt"
    processed_dir = "processed_data"
    os.makedirs(processed_dir, exist_ok=True)

    with open(input_dir, "r", encoding="utf-8") as f:
        text = f.read().replace(",", ".")

    clean_path = os.path.join(processed_dir, f"{i}kV-XRay_clean.txt")
    with open(clean_path, "w", encoding="utf-8") as f:
        f.write(text)

    data = np.loadtxt(clean_path, skiprows=2)
    x = data[:, 0]
    y = [int(i) for i in data[:, 1]]

    sx = np.full_like(x, 0.1)
    sy = np.sqrt(y)

    output = np.column_stack((x, y, sx, sy))

    processed_path = os.path.join(processed_dir, f"{i}kV-XRay_processed.txt")
    np.savetxt(processed_path, output, fmt="%.2e %.2f %.2e %.2f")

    os.remove(clean_path)

    print(f"Processed data saved at: {processed_path}")


Processed data saved at: processed_data/15kV-XRay_processed.txt
Processed data saved at: processed_data/18kV-XRay_processed.txt
Processed data saved at: processed_data/21kV-XRay_processed.txt
Processed data saved at: processed_data/24kV-XRay_processed.txt
Processed data saved at: processed_data/27kV-XRay_processed.txt
Processed data saved at: processed_data/30kV-XRay_processed.txt
Processed data saved at: processed_data/35kV-XRay_processed.txt


In [47]:
import ROOT
data = np.loadtxt("processed_data/35kV-XRay_processed.txt")
c = ROOT.TCanvas("c", "c", 1000, 600)
graph = ROOT.TGraphErrors("processed_data/35kV-XRay_processed.txt")
graph.SetTitle("35kV X-Ray Spectrum;Angle (deg);Rate (Imp/s)")

graph.Draw("A*")
c.Update()
c.SaveAs("35kV-XRay_plot.pdf")
del c, graph

Info in <TCanvas::Print>: pdf file 35kV-XRay_plot.pdf has been created


In [48]:

intervals = [
    (6.1,6.71),
    (7.4, 7.9),
    (13.2, 13.6),
    (14.75, 15.5),
    (19.9, 20.3),
    (22.4, 22.9),
    (30.4, 31.0)
]

c1 = ROOT.TCanvas("c1", "Canvas for 35kV", 1400, 900)
ROOT.gStyle.SetOptFit(0)
ROOT.gStyle.SetOptStat(0)

file_path = "processed_data/35kV-XRay_processed.txt"
graph = ROOT.TGraphErrors(file_path)
graph_points = graph.Clone("graph_points")
graph_errors = graph.Clone("graph_errors")

graph_points.SetMarkerStyle(20)
graph_points.SetMarkerSize(0.5)
graph_points.SetMarkerColor(ROOT.kBlue+2)

graph_errors.SetLineColor(ROOT.kGreen+2 ) 
graph_errors.SetMarkerColor(0)             
graph_errors.SetLineWidth(1)
graph_errors.SetMarkerStyle(0)

graph_points.Draw("AP") 
graph_errors.Draw("P SAME")      
graph_points.SetTitle("35kV X-Ray Spectrum;Angle (degrees);Rate (Imp/s)")
  

legend = ROOT.TLegend(0.60, 0.65, 0.82, 0.85)
legend.SetBorderSize(0)
legend.SetFillColor(ROOT.kWhite)
legend.SetFillStyle(1001)   
legend.SetTextFont(42)
legend.SetTextSize(0.025)

values, errors = [], []

for i, (xmin, xmax) in enumerate(intervals, start=1):
    fit_name = f"gauss_fit_{i}"
    gauss_fit = ROOT.TF1(fit_name, "[0]*TMath::Gaus(x,[1],[2])", xmin, xmax)
    gauss_fit.SetParameters(10, (xmin + xmax)/2, 0.5)
    gauss_fit.SetLineColor(ROOT.kRed) 

    gauss_fit.SetLineWidth(2)
    
    graph_errors.Fit(gauss_fit, "RQ+")
    
    mean = gauss_fit.GetParameter(1)
    sigma = gauss_fit.GetParameter(2)
    variance = sigma**2    

    values.append(mean)
    errors.append(variance)
    
    legend.AddEntry("", f"Peak{i}: \lambda = {mean:.2f} \pm {variance:.2f}", "n")

legend.Draw()
ROOT.gPad.SetTicks(1,1)
ROOT.gPad.SetGrid(1,1)

c1.Update()
c1.SaveAs("35kV-XRay_CalibrationPlot.pdf")
del c1, graph, gauss_fit, legend

Info in <TCanvas::Print>: pdf file 35kV-XRay_CalibrationPlot.pdf has been created


mean values of the peaks with their errors:

In [49]:
print(np.array([(values[i],errors[i]) for i in range(len(values))]))

[[6.65309798e+00 3.70922530e-01]
 [7.65421384e+00 2.86972833e-02]
 [1.33931374e+01 3.42471132e-02]
 [1.50760712e+01 3.58925470e-02]
 [2.00902001e+01 3.22328776e-02]
 [2.26670086e+01 2.86748740e-02]
 [3.06933187e+01 6.59578051e-02]]


bragg's law suggests: $n \lambda = 2 d \, \sin \theta$

we need $\lambda$ in order to obtain theoretical $\theta$

we can use $K_\alpha$ and $K_\beta$ values to obtain $\lambda$

from https://dlf.ug.edu.pl/wp-content/uploads/2014/03/X-Ray-Mo.pdf#page=2.17 , values are as follows:
$K_\alpha=17426.8 eV$ 
$K_\beta=19589.8 eV$

from Urusov, V.S., Blinov, V.V. Nature of the imperfection of NaCl-AgCl solid solutions. Russ Chem Bull 19, 231–234 (1970). https://doi.org/10.1007/BF00853362 , value for d is:

$d=282pm (10^-12m)$

In [50]:
h=6.6256e-34
C=2.9979e8
eV=1.6021e-19
d=282e-12
def wavelength(K):
    return h*C/(K*eV)
def tetha(n,wavelength):
    return float(np.degrees(np.arcsin(n*wavelength/(2*d))))


In [51]:
K_a=17426.8
K_b=19589.8
for i in range(3):
    print(f"Beta{i+1}")
    print(tetha(i+1,wavelength(K_b)))
    print(f"Alpha{i+1}")
    print(tetha(i+1,wavelength(K_a)))
print(values[:-1])    

Beta1
6.442907886667427
Alpha1
7.24664706373932
Beta2
12.969141041086196
Alpha2
14.612580550020782
Beta3
19.67225702627856
Alpha3
22.236018395775332
[6.653097984086529, 7.654213843506307, 13.393137403950941, 15.076071231880952, 20.090200114814536, 22.66700862373639]


In [52]:
cal_set=[]
cal_ab=[]
for i in range(3):
    cal_ab.append(tetha(i+1,wavelength(K_b)))
    cal_ab.append(tetha(i+1,wavelength(K_a))) 
cal_set.append(cal_ab)
cal_set.append(values[:-1])
cal_set.append(errors[:-1])#i will ditch the last data point since it is missing the beta pair and unreliable

In [53]:
cal_set

[[6.442907886667427,
  7.24664706373932,
  12.969141041086196,
  14.612580550020782,
  19.67225702627856,
  22.236018395775332],
 [6.653097984086529,
  7.654213843506307,
  13.393137403950941,
  15.076071231880952,
  20.090200114814536,
  22.66700862373639],
 [0.3709225297623251,
  0.02869728326878947,
  0.03424711319713821,
  0.035892546958856686,
  0.032232877634078554,
  0.028674874049479493]]

In [54]:
print([i*10 for i in cal_set[2]])

[3.709225297623251, 0.2869728326878947, 0.3424711319713821, 0.35892546958856686, 0.32232877634078555, 0.28674874049479493]


In [55]:
import ROOT
ROOT.gStyle.SetOptFit(1)
# Data
y = cal_set[0]
x = cal_set[1]
sx = cal_set[2]
sy = [0]*6  

graph = ROOT.TGraphErrors(len(x))
for i in range(len(x)):
    graph.SetPoint(i, x[i], y[i])
    graph.SetPointError(i, sx[i], sy[i])

fit_func = ROOT.TF1("fit_func", "[0] + [1]*x", min(x), max(x))
fit_func.SetParameters(0, 1)  # initial guesses

graph.Fit(fit_func)
cal_parameters=[(fit_func.GetParameter(i),fit_func.GetParError(i)) for i in range(2)]

visual_scale_factor =20
starting_point=2
print(f"Applying a visual scaling factor of {visual_scale_factor} to points {starting_point} to {len(x)}'s error bars for drawing.")

for i in range(1,len(x)):
    graph.SetPointError(i, sx[i] * visual_scale_factor, sy[i])
graph.SetMarkerStyle(20)
graph.SetMarkerColor(ROOT.kBlue)
graph.SetTitle("Calibration Function;#theta_{exp};#theta_{thr}")

c = ROOT.TCanvas("c", "Line Fit", 800, 600)
graph.Draw("AP")
fit_func.Draw("same")
c.Update()
stats = graph.GetListOfFunctions().FindObject("stats")
if stats:
    stats.SetX1NDC(0.15)  # lower-left x (0 to 1)
    stats.SetY1NDC(0.75)  # lower-left y
    stats.SetX2NDC(0.45)  # upper-right x
    stats.SetY2NDC(0.87)   # upper-right y
    stats.SetTextSize(0.03)
    c.Modified()

ROOT.gPad.SetTicks(1, 1)
ROOT.gPad.SetGrid(1, 1)
c.Update()
c.SaveAs("cal_function.pdf")
del c, graph, fit_func


Applying a visual scaling factor of 20 to points 2 to 6's error bars for drawing.
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      1.72262
NDf                       =            4
Edm                       =  2.77698e-06
NCalls                    =           45
p0                        =    -0.408075   +/-   0.0416658   
p1                        =     0.998842   +/-   0.00248468  


Info in <TCanvas::Print>: pdf file cal_function.pdf has been created


In [56]:
cal_parameters

[(-0.40807496549392136, 0.04166581435994885),
 (0.9988420932493778, 0.0024846833469566666)]

In [57]:
m=cal_parameters[1][0]
b=cal_parameters[0][0]
s_m=cal_parameters[1][1]
s_b=cal_parameters[0][1]
def calerror(tetha):
    return float(np.sqrt((s_m*tetha)**2+(m*0.1)**2+s_b**2))

def calibrate(tetha):
    m=cal_parameters[1][0]
    b=cal_parameters[0][0]
    return tetha*m+b

In [58]:
print(calibrate(values[2]),calerror(values[2]))

12.969554434244973 0.11322676821391196


At this point we obtained the calibration function and can convert our experimental tetha values with their errors to the theoreticly correct scale. we need to update our datasets on 15,18,21,24,27,30keV accordingly. 

In [59]:
for i in [15,18,21,24,27,30]:
    input_dir = f"/home/master/Desktop/443exps/x-ray/processed_data/{i}kV-XRay_processed.txt"
    processed_dir = "calibrated_data"
    os.makedirs(processed_dir, exist_ok=True)

    with open(input_dir, "r", encoding="utf-8") as f:
        text = f.read().replace(",", ".")

    clean_path = os.path.join(processed_dir, f"{i}kV-XRay_clean.txt")
    with open(clean_path, "w", encoding="utf-8") as f:
        f.write(text)

    data = np.loadtxt(clean_path)
    x = data[:, 0]
    y = data[:, 1]

    sx = np.full_like(x, 0.1)
    sy = np.sqrt(y)
    cal_x=[calibrate(i) for i in x]
    cal_sx=[calerror(i) for i in x]
    output = np.column_stack((cal_x, y,cal_sx, sy))

    processed_path = os.path.join(processed_dir, f"{i}kV-XRay_calibrated.txt")
    np.savetxt(processed_path, output, fmt="%.2e %.2f %.2e %.2f")

    os.remove(clean_path)

    print(f"Processed data saved at: {processed_path}")


Processed data saved at: calibrated_data/15kV-XRay_calibrated.txt
Processed data saved at: calibrated_data/18kV-XRay_calibrated.txt
Processed data saved at: calibrated_data/21kV-XRay_calibrated.txt
Processed data saved at: calibrated_data/24kV-XRay_calibrated.txt
Processed data saved at: calibrated_data/27kV-XRay_calibrated.txt
Processed data saved at: calibrated_data/30kV-XRay_calibrated.txt


we finaly need to convert them into wavelength using braggs law. We shall assuma n=1 since our dataset does not reach to level of alpha1 or betha1 peaks.

In [60]:
def braggs(tetha):
    return np.sin(np.deg2rad(tetha))*2*d

In [61]:
braggs(11.78)

np.float64(1.1514305351992426e-10)

In [62]:
def err_braggs(tetha,err_tetha):
    return np.cos(np.deg2rad(tetha))*2*d*np.deg2rad(err_tetha)


In [63]:
braggs(11.78)

np.float64(1.1514305351992426e-10)

In [64]:
err_braggs(11.78,0.11)

np.float64(1.059997057213857e-12)

In [65]:
for i in [15,18,21,24,27,30]:
    input_dir = f"/home/master/Desktop/443exps/x-ray/calibrated_data/{i}kV-XRay_calibrated.txt"
    processed_dir = "wave_data"
    os.makedirs(processed_dir, exist_ok=True)

    with open(input_dir, "r", encoding="utf-8") as f:
        text = f.read().replace(",", ".")

    clean_path = os.path.join(processed_dir, f"{i}kV-XRay_clean.txt")
    with open(clean_path, "w", encoding="utf-8") as f:
        f.write(text)

    data = np.loadtxt(clean_path)
    x = data[:, 0]
    y = data[:, 1]

    sx = data[:,2]
    sy = data[:,3]
    cal_x=[braggs(i) for i in x]
    cal_sx=[err_braggs(x[i],sx[i]) for i in range(len(x))]
    output = np.column_stack((cal_x, y,cal_sx, sy))

    processed_path = os.path.join(processed_dir, f"{i}kV-XRay_lambda.txt")
    np.savetxt(processed_path, output, fmt="%.2e %.2e %.2e %.2e")

    os.remove(clean_path)

    print(f"Processed data saved at: {processed_path}")

Processed data saved at: wave_data/15kV-XRay_lambda.txt
Processed data saved at: wave_data/18kV-XRay_lambda.txt
Processed data saved at: wave_data/21kV-XRay_lambda.txt
Processed data saved at: wave_data/24kV-XRay_lambda.txt
Processed data saved at: wave_data/27kV-XRay_lambda.txt
Processed data saved at: wave_data/30kV-XRay_lambda.txt


we now can fit a line to linear segments of datasets. we need the intervals for each

In [66]:
L=[(15,(8e-11,9e-11)),(18,(6.50e-11,8e-11)),
   (21,(5.80e-11,7.20e-11)),(24,(5.7e-11,7e-11)),
   (27,(6e-11,7e-11)),(30,(6e-11,7e-11))]

In [67]:
ROOT.gROOT.SetBatch(True)
L=[(15,(8e-11,9e-11)),(18,(6.50e-11,7.8e-11)),
   (21,(5.80e-11,7.20e-11)),(24,(5.60e-11,6.7e-11)),
   (27,(5.4e-11,6.5e-11)),(30,(5.3e-11,6.4e-11))]
intercept_params=[]
for idx, (kv, (fit_a, fit_b)) in enumerate(L):
    c = ROOT.TCanvas(f"c{idx}", f"{kv}kV", 1000, 600)
    
    data = np.loadtxt(f"wave_data/{kv}kV-XRay_lambda.txt")
    x = data[:,0]
    y = data[:,1]
    sx = data[:,2]
    sy = data[:,3]

    graph = ROOT.TGraphErrors(len(x))
    for i in range(len(x)):
      graph.SetPoint(i, x[i], y[i])
      graph.SetPointError(i, sx[i], sy[i])
    graph.SetTitle(f"{kv}kV X-Ray Spectrum ;Wavelength (m);Rate (Imp/s)")
    graph.SetMarkerStyle(20)
    graph.SetMarkerSize(0.7)
    graph.SetMarkerColor(ROOT.kBlue +1)
    graph.Draw("AP")
    fit = ROOT.TF1("line_fit", "[0] + [1]*x",fit_a,fit_b)
    fit.SetParameter(0, 0.0)         
    fit.SetParameter(1, 1e11)        
    fit_result = ROOT.TFitResultPtr(graph.Fit(fit, "RSQ"))
    cov_ab = fit_result.CovMatrix(0, 1)  # i ditch this if i need
    var_a  = fit_result.CovMatrix(1, 1)  # variance of slope 
    var_b  = fit_result.CovMatrix(0, 0)  # variance of intercept

# Convert to standard deviations
    sa = float(np.sqrt(var_a))
    sb = float(np.sqrt(var_b))

    a = fit.GetParameter(1)
    b = fit.GetParameter(0)
    intercept_params.append([a,b,sa,sb,cov_ab])#cov_ab here
    x_intercept = -b / a
    print(f"{kv}kV x-intercept: {x_intercept:.3e} m")
    y_max = (fit_b+0.5e-11)*a+b
    xline = ROOT.TLine(x_intercept, 0, fit_b+0.5e-11, y_max) 
    xline.SetLineColor(ROOT.kBlue)
    xline.SetLineStyle(2)
    xline.SetLineWidth(2)
    xline.Draw("same")

    legend = ROOT.TLegend(0.1, 0.75, 0.50, 0.85) 
    legend.SetBorderSize(0)
    legend.SetFillStyle(0)
    legend.AddEntry("",f"x-intercept = {x_intercept:.3e} m","n")
    legend.Draw("same")
    c.Update()
    c.SaveAs(f"{kv}kV_lambda_intercept.pdf")
    del c, graph, fit, xline, legend

15kV x-intercept: 7.962e-11 m
18kV x-intercept: 6.520e-11 m
21kV x-intercept: 5.605e-11 m
24kV x-intercept: 5.152e-11 m
27kV x-intercept: 4.602e-11 m
30kV x-intercept: 4.007e-11 m


Info in <TCanvas::Print>: pdf file 15kV_lambda_intercept.pdf has been created
Info in <TCanvas::Print>: pdf file 18kV_lambda_intercept.pdf has been created
Info in <TCanvas::Print>: pdf file 21kV_lambda_intercept.pdf has been created
Info in <TCanvas::Print>: pdf file 24kV_lambda_intercept.pdf has been created
Info in <TCanvas::Print>: pdf file 27kV_lambda_intercept.pdf has been created
Info in <TCanvas::Print>: pdf file 30kV_lambda_intercept.pdf has been created


In [68]:
intercept_params

[[2277311570637.839,
  -181.3235257630652,
  422713420649.77045,
  35.451041376793206,
  -14977222790094.352],
 [3006573409271.2686,
  -196.03729727597687,
  405162728010.21545,
  28.41510485536157,
  -11499029714546.826],
 [3991413647418.495,
  -223.73461345127373,
  482062712159.1669,
  30.858146127961025,
  -14847807325466.254],
 [6919821163568.986,
  -356.4863003410447,
  1146646396893.9426,
  70.20839956390087,
  -80413590534696.36],
 [7367472264241.144,
  -339.03235645760947,
  1086039714303.459,
  63.883396816582426,
  -69269165747393.28],
 [7579114565023.897,
  -303.7109936916751,
  1284757216703.0935,
  74.47545187719386,
  -95538169573427.33]]

In [69]:
def intercept_err(parameters):
    a=parameters[0]
    b=parameters[1]
    sa=parameters[2]
    sb=parameters[3]
    cov_ab=parameters[4]
    sigma=float(np.sqrt((sb/a)**2 + (b*sa/a**2)**2 - 2*b*cov_ab/(a**3))) #- 2*b*cov_ab/(a**3) here if you remove
    intercept=float(-b/a)
    return(intercept,sigma)
    

In [70]:
final_data=[intercept_err(i) for i in intercept_params]

In [71]:
final_data

[(7.962174702000893e-11, 9.373708604120842e-13),
 (6.520289731541671e-11, 7.994539634542902e-13),
 (5.6053978167854726e-11, 1.0579425216861873e-12),
 (5.151669268822293e-11, 1.6689071568384538e-12),
 (4.601745948921195e-11, 1.9366678942442875e-12),
 (4.00720943173548e-11, 3.0667655024373124e-12)]

we can finaly move on to calculating energies from our base wavelength. Base wavelength corresponds to the higer energy of x-rays which is given with:

$Energy:eV=\frac{hc}{\lambda}$

where V is accelerating potential and e is fundamental charge of electron.we ahould first convert $\lambda$ into $f=\frac{c}{\lambda}$




In [72]:
def frequency_err(l):
    f=C/l[0]
    sf=f*l[1]/l[0]
    return (f,sf)


In [73]:
f_data=np.array([frequency_err(i) for i in final_data])

In [74]:
f_data

array([[3.76517737e+18, 4.43266781e+16],
       [4.59780182e+18, 5.63737355e+16],
       [5.34823771e+18, 1.00940705e+17],
       [5.81927885e+18, 1.88518238e+17],
       [6.51470123e+18, 2.74174473e+17],
       [7.48126608e+18, 5.72550278e+17]])

now we need to plot energy vs.frequency and investigate the slope  

In [75]:
e_list=[]
for i in [15,18,21,24,27,30]:
    e_list.append(i*1e3*eV)

In [76]:
e_list

[2.40315e-15, 2.88378e-15, 3.36441e-15, 3.84504e-15, 4.32567e-15, 4.8063e-15]

In [77]:
y = e_list
x = f_data[:,0]
sx = f_data[:,1]
sy = [0]*6 

final solution: i implamented covariance to llambda to make it more precise and scaled the final e vs. f graph to help with regression. 

general roadmap of this analysis as follows (in case i forget):

1-fit gaussians to the K_alpha and K-Beta peaks in 35keV data.

2-calculate theoretical peaks using braggs law and theoretical values for alpha and beta(you can see the reference to that up sommewhere)

3-find linear relation theoretical=a*experimental+b

4-calibrate all datasets and convert tehta values to wavelength using braggs law combined with E=hc/lambda

5-plot the datasets and find intercept of the linear part of the e vs. lambda graph

6-this gives you min lambda that bremsstraghlung event can produce. which gives the max energy output

7-find frequency of that lambda_min and plot with energy

8-slope should give h from E=hf (Duane-hunt)    

In [78]:
def rescale_f(freq):
    f=freq[:,0]*1e-18
    sf=freq[:,1]*1e-18
    return np.array([(float(f[i]),float(sf[i])) for i in range(len(f))])

def rescale_e(energy):
    return [float(energy[i]*1e16) for i in range(len(energy))]



In [79]:
rescale_f(f_data)
rescale_e(e_list)

[24.0315, 28.8378, 33.6441, 38.4504, 43.2567, 48.063]

In [80]:
import ROOT
ROOT.gStyle.SetOptFit(1)

y = rescale_e(e_list)
f_scaled=rescale_f(f_data)
x = f_scaled[:,0]
sx = f_scaled[:,1]
sy = [0]*6 

graph = ROOT.TGraphErrors(len(x))
for i in range(len(x)):
    graph.SetPoint(i, x[i], y[i])
    graph.SetPointError(i, sx[i], sy[i])

fit_func = ROOT.TF1("fit_func", "[0] + [1]*x")
b_guess = (y[-1] - y[0]) / (x[-1] - x[0])
a_guess = y[0] - b_guess * x[0]

fit_func.SetParameters(a_guess, b_guess)
print("Initial guesses:", a_guess, b_guess)

graph.Fit(fit_func,"S")
cal_parameters=[(fit_func.GetParameter(i),fit_func.GetParError(i)) for i in range(2)]

graph.SetMarkerStyle(20)
graph.SetMarkerColor(ROOT.kBlue)
graph.SetTitle("Experimental Planck Constant;f_{max};Energy (J)")

c = ROOT.TCanvas("c", "Line Fit", 800, 600)
graph.Draw("AP")
fit_func.Draw("same")
c.Update()
stats = graph.GetListOfFunctions().FindObject("stats")
if stats:
    stats.SetX1NDC(0.15)  
    stats.SetY1NDC(0.75)  
    stats.SetX2NDC(0.45) 
    stats.SetY2NDC(0.87)   
    stats.SetTextSize(0.03)
    c.Modified()

ROOT.gPad.SetTicks(1, 1)
ROOT.gPad.SetGrid(1, 1)
c.Update()
c.SaveAs("h_plot.pdf")

Initial guesses: -0.3174504655722643 6.4668800632860455
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      4.04913
NDf                       =            4
Edm                       =  2.50958e-07
NCalls                    =           41
p0                        =    -0.516195   +/-   1.34503     
p1                        =      6.46686   +/-   0.308572    


Info in <TCanvas::Print>: pdf file h_plot.pdf has been created


In [81]:
cal_parameters

[(-0.516194702392851, 1.3450294242416538),
 (6.4668625112129074, 0.3085720588544602)]

In [82]:
result=[cal_parameters[1][i]*1e-34 for i in range(2)]
print(f"planck constant = {result[0]} +- {result[1]}")

planck constant = 6.466862511212907e-34 +- 3.085720588544602e-35
